Implementing an Auotoencoder network for anomaly detection based on the proposed procedure in following paper:
Collision Detection for Robot Manipulators Using Unsupervised Anomaly Detection Algorithms Kyu Min Park , Younghyo Park , Sangwoong Yoon , and Frank C. Park , Fellow, IEEE



In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from IPython.core.interactiveshell import InteractiveShell

from sklearn.preprocessing import MinMaxScaler
import keras 
from keras import layers


Now reload data from data set 

In [3]:
data_df = pd.read_csv("Data sets/dataCircularPath_f15N.csv")
data_df.head()

,cartForce1_X,cartForce1_Y,cartForce1_Z,cartTorque1_TauX,cartTorque1_TauY,cartTorque1_TauZ,CartPosMsr1_X,CartPosMsr1_Y,CartPosMsr1_Z,CollisionType
0,-0.198948,-0.350991,1.579013,-0.298485,-0.309085,-0.298117,0.057388,700.068897,60.028683,NoColl
1,-0.200875,-0.351639,1.580461,-0.298425,-0.309606,-0.298192,0.057364,700.068860,60.028681,NoColl
2,-0.192838,-0.348998,1.572937,-0.298804,-0.307639,-0.297851,0.057515,700.068878,60.028637,NoColl
3,-0.187162,-0.355347,1.579814,-0.297828,-0.306840,-0.297809,0.057514,700.068828,60.028644,NoColl
4,-0.202570,-0.363153,1.573990,-0.296778,-0.311663,-0.298496,0.057417,700.068777,60.028608,NoColl


Sampling signals into 60ms window with an Interval of 4ms.
remember, that the kuka sampling rate set to 500Hz. So I took the mean of each 2 rows. But first, pick the non-collision data and remove the last column 

In [4]:
data_df_nocoll =  data_df[data_df['CollisionType'].str.contains('NoColl')]
data_df_nocoll = data_df_nocoll.drop(["CollisionType"],axis=1) #dropping cols with string or integer value
data_df_nocoll = data_df_nocoll.astype('float16')
data_df_nocoll.shape


(180000, 9)

In [5]:

col_names = data_df_nocoll.columns
data_df_nocoll = pd.DataFrame(data_df_nocoll.values.reshape(-1,2,data_df_nocoll.shape[1]).mean(1),columns=data_df_nocoll.columns) 
data_df_nocoll.columns = col_names
data_df_nocoll.head()
#data_df_nocoll.columns


,cartForce1_X,cartForce1_Y,cartForce1_Z,cartTorque1_TauX,cartTorque1_TauY,cartTorque1_TauZ,CartPosMsr1_X,CartPosMsr1_Y,CartPosMsr1_Z
0,-0.199951,-0.351318,1.580078,-0.298340,-0.309326,-0.298096,0.057373,700.0,60.03125
1,-0.189941,-0.352051,1.576172,-0.298340,-0.307129,-0.297852,0.057526,700.0,60.03125
2,-0.201904,-0.360352,1.577148,-0.297119,-0.311523,-0.298340,0.057434,700.0,60.03125
3,-0.163818,-0.350830,1.550781,-0.298584,-0.302246,-0.296875,0.058472,700.0,60.03125
4,0.058105,-0.330078,1.344727,-0.305176,-0.255859,-0.290527,0.065796,700.0,60.03125


Now, each 2 row are 4ms apart that means 15 of them becomes a 60ms window.
In this case, R(6)*(15+1) : 6 is the cart. F/T (in paper its 4 bc of choosing joints 1 to 4)


In [6]:
# First scale data in range 0-1
dataset = data_df_nocoll.values
scaler = MinMaxScaler(feature_range=(0, 1)) # creates an object which scales! (scal er)
dataset = scaler.fit_transform(dataset)

# split into train and test sets
# in case of Auoto encoder, test data output must result in zero anomaly
train_size = int(len(dataset) * 0.80)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
print(len(train), len(test))

72000 18000


Now, creating the input vector R as below:
R[ 1 - 16]
R[ 2 - 17]
R[ 3 - 18] and so on...

In [7]:
def create_dataset(dataset, look_back=1):
 dataX, dataY = [], []
 for i in range(len(dataset)-look_back-1):
    a = dataset[i:(i+look_back), :6] # keeping only 6dof F/T
    dataX.append(a)
    #dataY.append(dataset[i + look_back, 0]) # AE uses the same input and output
 return np.array(dataX)#, np.array(dataY)

N = int(60/4)
trainR = create_dataset(train, N+1)
testR = create_dataset(test, N+1)
print(trainR[:2])

[[[0.5312  0.432   0.1103  0.322   0.549   0.3157 ]
  [0.5317  0.432   0.1102  0.322   0.55    0.316  ]
  [0.531   0.4316  0.1102  0.3225  0.548   0.3154 ]
  [0.5327  0.432   0.1095  0.3218  0.552   0.317  ]
  [0.5444  0.4329  0.104   0.319   0.5693  0.3228 ]
  [0.567   0.4358  0.0928  0.312   0.6     0.3313 ]
  [0.6055  0.4407  0.078   0.3013  0.6543  0.3474 ]
  [0.64    0.4473  0.065   0.289   0.696   0.3591 ]
  [0.6606  0.4514  0.0636  0.2822  0.718   0.3657 ]
  [0.6646  0.4514  0.07367 0.2803  0.7285  0.3716 ]
  [0.637   0.4438  0.0904  0.2883  0.7     0.3667 ]
  [0.58    0.4316  0.1134  0.3025  0.6406  0.3535 ]
  [0.537   0.4167  0.1423  0.3198  0.609   0.349  ]
  [0.525   0.3972  0.1724  0.3413  0.6167  0.3535 ]
  [0.5225  0.378   0.1985  0.3608  0.6294  0.354  ]
  [0.506   0.3652  0.2205  0.3745  0.619   0.344  ]]

 [[0.5317  0.432   0.1102  0.322   0.55    0.316  ]
  [0.531   0.4316  0.1102  0.3225  0.548   0.3154 ]
  [0.5327  0.432   0.1095  0.3218  0.552   0.317  ]
  [0.5444 

In [8]:
# Reshape the r vectors into x in the paper
( num,rows_R ,cols_R) = trainR.shape
trainX = trainR.reshape(-1,rows_R * cols_R)
testX = testR.reshape(-1,rows_R*cols_R)
input_size_X = rows_R * cols_R
print(trainX[:2])

[[0.5312  0.432   0.1103  0.322   0.549   0.3157  0.5317  0.432   0.1102
  0.322   0.55    0.316   0.531   0.4316  0.1102  0.3225  0.548   0.3154
  0.5327  0.432   0.1095  0.3218  0.552   0.317   0.5444  0.4329  0.104
  0.319   0.5693  0.3228  0.567   0.4358  0.0928  0.312   0.6     0.3313
  0.6055  0.4407  0.078   0.3013  0.6543  0.3474  0.64    0.4473  0.065
  0.289   0.696   0.3591  0.6606  0.4514  0.0636  0.2822  0.718   0.3657
  0.6646  0.4514  0.07367 0.2803  0.7285  0.3716  0.637   0.4438  0.0904
  0.2883  0.7     0.3667  0.58    0.4316  0.1134  0.3025  0.6406  0.3535
  0.537   0.4167  0.1423  0.3198  0.609   0.349   0.525   0.3972  0.1724
  0.3413  0.6167  0.3535  0.5225  0.378   0.1985  0.3608  0.6294  0.354
  0.506   0.3652  0.2205  0.3745  0.619   0.344  ]
 [0.5317  0.432   0.1102  0.322   0.55    0.316   0.531   0.4316  0.1102
  0.3225  0.548   0.3154  0.5327  0.432   0.1095  0.3218  0.552   0.317
  0.5444  0.4329  0.104   0.319   0.5693  0.3228  0.567   0.4358  0.0928
  0.

In [9]:
# forming AE 
input_vec_x = keras.Input(shape=(input_size_X, ))
encoder1 = layers.Dense(48, activation='relu')(input_vec_x)
bottlneck = layers.Dense(8,activation='relu')(encoder1)
decoder1 = layers.Dense(48,activation='relu')(bottlneck)
output_layer = layers.Dense(input_size_X,activation='linear')(decoder1)
autoencoder1 = keras.Model(inputs = input_vec_x,outputs = output_layer)

autoencoder1.compile(optimizer='adam',loss='mae')
print(autoencoder1.summary())


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 96)]              0         
                                                                 
 dense (Dense)               (None, 48)                4656      
                                                                 
 dense_1 (Dense)             (None, 8)                 392       
                                                                 
 dense_2 (Dense)             (None, 48)                432       
                                                                 
 dense_3 (Dense)             (None, 96)                4704      
                                                                 
Total params: 10,184
Trainable params: 10,184
Non-trainable params: 0
_________________________________________________________________
None


In [10]:
# train model
history = autoencoder1.fit(trainX,trainX,batch_size=100,epochs=20,verbose=2)

Epoch 1/20
720/720 - 1s - loss: 0.0555 - 1s/epoch - 2ms/step
Epoch 2/20
720/720 - 1s - loss: 0.0206 - 837ms/epoch - 1ms/step
Epoch 3/20
720/720 - 1s - loss: 0.0195 - 840ms/epoch - 1ms/step
Epoch 4/20
720/720 - 1s - loss: 0.0189 - 862ms/epoch - 1ms/step
Epoch 5/20
720/720 - 1s - loss: 0.0186 - 860ms/epoch - 1ms/step
Epoch 6/20
720/720 - 1s - loss: 0.0185 - 819ms/epoch - 1ms/step
Epoch 7/20
720/720 - 1s - loss: 0.0184 - 826ms/epoch - 1ms/step
Epoch 8/20
720/720 - 1s - loss: 0.0183 - 816ms/epoch - 1ms/step
Epoch 9/20
720/720 - 1s - loss: 0.0183 - 835ms/epoch - 1ms/step
Epoch 10/20
720/720 - 1s - loss: 0.0182 - 851ms/epoch - 1ms/step
Epoch 11/20
720/720 - 1s - loss: 0.0181 - 909ms/epoch - 1ms/step
Epoch 12/20
720/720 - 1s - loss: 0.0181 - 820ms/epoch - 1ms/step
Epoch 13/20
720/720 - 1s - loss: 0.0180 - 807ms/epoch - 1ms/step
Epoch 14/20
720/720 - 1s - loss: 0.0180 - 821ms/epoch - 1ms/step
Epoch 15/20
720/720 - 1s - loss: 0.0180 - 810ms/epoch - 1ms/step
Epoch 16/20
720/720 - 1s - loss: 0.01

In [18]:

testXhat_predict = autoencoder1.predict(testX)
trainXhat_predict = autoencoder1.predict(trainX)


2250/2250 [==============================] - 2s 869us/step


In [23]:
# Plot testX and testXhat to check the difference
# remember that in AE target (y) is the same as input (x)

# first reshape from network input/output vector(-1,96,1) into R dimension (-1,16,6)
testXhat_predict_Rshape = testXhat_predict.reshape(-1,16,6)
trainXhat_predict_Rshape = trainXhat_predict.reshape(-1,16,6) 
# now revert back the R to the time series original data shape
testXhat_predict_data = trainXhat_predict_Rshape[:,0,:]
dumyydata = np.zeros()
testXhat_predict_data = scaler.inverse_transform(testXhat_predict_data,)
#trainXhat_predict_Rshape = 
#testXhat_predict = scaler.inverse_transform(testXhat_predict)----------:,1,1
#trainXhat_predict = scaler.inverse_transform(trainXhat_predict)
fig, ax = plt.subplots()
#ax.plot(  testX_R, width=1, edgecolor="white", linewidth=0.7)
plt.show()

ValueError: operands could not be broadcast together with shapes (71983,6) (9,) (71983,6) 